In [1]:
import plotly
import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

import pandas

from util import ESConnection
from elasticsearch_dsl import Search, Q

es_conn = ESConnection()

init_notebook_mode(connected=True)

INIT_DATE_4Y = '2013-01-01'
INIT_DATE_1Y = '2016-04-01'
END_DATE = '2017-04-01'

## Basic metrics to be in the report:

**Git**
* Evolution and trends over time [per quarter] of commits by gender
  * Commits by gender (columns: hash, gender)
* Evolution and trends [per quarter] of developers over time by gender
  * Developers by gender (columns: name, uuid, gender)
* Evolution and trends of type of contributions (code or others) by gender over time
  * Type of file touched by developers (columns: filetype, gender)

**Gerrit**
* Evolution of code reviews over time by gender
  * Count votes by gender (vote, gender)
* Evolution of code reviews developers over time by gender
  * Count people voting (name, uuid, vote)
* Evolution of core reviews over time by gender
  * Votes +2 or -2 (vote +2/-2 and gender)
* Evolution of core review developers over time by gender
  * People voting +2 or -2 (name, uuid, vote +2/-2, gender)

**Others**
* Evolution of attracted developers over time by gender
  * First commit by gender
* Time working in the community by gender
  * Time difference between the first and last contribution by all developers (so how long developers remain in OpenStack?).

# SOME FUNCTIONS

In [2]:
def query_metric_over_time(index, metric_name, metric_field, filters = []):
    
    s = Search(using=es_conn, index=index)  # Index selection
    for filtering in filters:
        s = s.filter(filtering)
    s.aggs.bucket('gender', 'terms', field='gender')\
          .bucket('time', 'date_histogram', field='date', interval='quarter')\
          .metric(metric_name, 'cardinality', field=metric_field, precision_threshold=10000)
    result = s.execute()

    value = result.to_dict()["aggregations"]['gender']['buckets']
    
    df = pandas.DataFrame()
    for i in value:
        df2 = (pandas.DataFrame.from_dict(i["time"]["buckets"]))
        df2["gender"] = i["key"]
        df2[metric_name] = df2[metric_name].apply(lambda row:row["value"])
        df = pandas.concat([df, df2])
        
    return df

In [53]:
def query_total_piechart(index, metric_name, metric_field, filters = []):
    s = Search(using=es_conn, index=index)  # Index selection
    for filtering in filters:
        s = s.filter(filtering)
    #s = s.filter('range', date={'gt': start_date, 'lt':'now/M'}) # filter date
    s.aggs.bucket('gender', 'terms', field='gender')\
          .metric(metric_name, 'cardinality', field=metric_field, precision_threshold=1000000)
    result = s.execute()
    
    buckets = result.to_dict()["aggregations"]["gender"]["buckets"]
    pie_chart_labels = []
    pie_chart_values = []
    for bucket in buckets:
        pie_chart_labels.append(bucket["key"])
        pie_chart_values.append(bucket[metric_name]["value"])
        
    return pie_chart_labels, pie_chart_values

In [4]:
def draw_evolutionary_chart(label, values):
    female_changesets = plotly.graph_objs.Scatter(
    x = df[df["gender"]=="female"][label],
    y = df[df["gender"]=="female"][values],
    name = 'female'
    )

    male_changesets = plotly.graph_objs.Scatter(
        x = df[df["gender"]=="male"][label],
        y = df[df["gender"]=="male"][values],
        name = 'male'
    )

    unknown_changesets = plotly.graph_objs.Scatter(
        x = df[df["gender"]=="NotKnown"][label],
        y = df[df["gender"]=="NotKnown"][values],
        name = 'unknown'
    )

    iplot([female_changesets, male_changesets, unknown_changesets])
    # Using only plot and not iplot creates and HTML page with JS to play with the chart

In [5]:
def draw_piechart(pie_chart_labels, pie_chart_values, title):
            
    fig = {
    'data': [{'labels': pie_chart_labels,
              'values': pie_chart_values,
              'type': 'pie'}],
    'layout': {'title': title}
     }
    iplot(fig)

# GERRIT

In [49]:
INDEX = "gerrit_eventized"
filter_date_4y = Q('range', date={'gte': INIT_DATE_4Y, 'lt': END_DATE})
filter_date_1y = Q('range', date={'gt': INIT_DATE_1Y, 'lt': END_DATE})


## Changeset Submissions by Gender

### Evolution of submissions sent over time by gender


In [50]:
METRIC_NAME = "changesets"
METRIC_FIELD = "id"
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y])

In [8]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

### Aggregated changeset submissions by gender

In [9]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y])

In [10]:
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submissions by Gender (last 4 years)')

In [54]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_1y])


In [55]:
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submissions by Gender (last year)')

## Population of people submitting changesets
### Evolution of submitters over time by gender
  * Count people submitting (id, uuid)

In [13]:
METRIC_NAME = "submitters"
METRIC_FIELD = "uuid"
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y])
draw_evolutionary_chart("key_as_string", METRIC_NAME)

### Aggregated number of submitters


In [14]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y])
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submitters by Gender (last 4 years)')

In [15]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD,  [filter_date_1y])
draw_piechart(pie_chart_labels, pie_chart_values, 'Changeset Submitters by Gender (last year)')

* Evolution of code reviews developers over time by gender
  * Count people voting (name, uuid, vote)


## Number of votes by gender

In [16]:
METRIC_NAME = "reviewer"
METRIC_FIELD = "uuid"
filter_vote = Q('term', eventtype='CHANGESET_PATCHSET_APPROVAL_Code-Review') # filter by event: vote a code review

df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_vote])
draw_evolutionary_chart("key_as_string", "doc_count")

## Number of people voting

In [17]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

## Number of core reviews (-2 OR +2) by gender

In [18]:
METRIC_NAME = "core_reviewers"
METRIC_FIELD = "uuid"

filter_core_vote = Q('terms', value=["2", "-2"])
filter_vote = Q('term', eventtype='CHANGESET_PATCHSET_APPROVAL_Code-Review') # filter by event: vote a code review
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_core_vote, filter_date_4y, filter_vote])
draw_evolutionary_chart("key_as_string", "doc_count")

## Number of people acting as core reviews (-2 OR +2) by gender

In [19]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

# GIT

In [20]:
INDEX = "git_eventized"
filter_merges_addedlines = Q('range', addedlines={'gt': 0})
filter_merges_removedlines = Q('range', removedlines={'gt': 0})
filter_bots = Q('bool', must_not=[Q('match', gender_analyzed_name='Jenkins')])

In [38]:
git_filter = Q('bool',
    must_not=[Q('match', gender_analyzed_name='Jenkins')],
    should=[filter_merges_addedlines, filter_merges_removedlines]
)

* Evolution and trends over time [per quarter] of commits by gender
  * Commits by gender (columns: hash, gender)

In [21]:
METRIC_NAME = "commits"
METRIC_FIELD = "id.keyword"
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_merges_addedlines,
                                                               filter_merges_removedlines, filter_bots])

In [22]:
draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [23]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, filter_merges_addedlines,
                                                             filter_merges_removedlines, filter_bots])
draw_piechart(pie_chart_labels, pie_chart_values, 'Commits by Gender (last 4 years)')

In [39]:
pie_chart_labels_2, pie_chart_values_2 = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, git_filter])
draw_piechart(pie_chart_labels_2, pie_chart_values_2, 'Commits by Gender (last 4 years)')

In [24]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, filter_merges_addedlines,
                                                             filter_merges_removedlines, filter_bots])
draw_piechart(pie_chart_labels, pie_chart_values, 'Commits by Gender (last year)')

In [40]:
pie_chart_labels_2, pie_chart_values_2 = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, git_filter])
draw_piechart(pie_chart_labels_2, pie_chart_values_2, 'Commits by Gender (last year)')


* Evolution and trends [per quarter] of developers over time by gender
  * Developers by gender (columns: name, uuid, gender)

In [25]:
METRIC_NAME = "authors"
METRIC_FIELD = "uuid"
filter_merges_addedlines = Q('range', addedlines={'gt': 0})
filter_merges_removedlines = Q('range', removedlines={'gt': 0})
filter_bots = Q('bool', must_not=[Q('match', gender_analyzed_name='Jenkins')])
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_merges_addedlines, filter_merges_removedlines, filter_bots])

draw_evolutionary_chart("key_as_string", METRIC_NAME)

In [26]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_4y, filter_merges_addedlines,
                                                             filter_merges_removedlines, filter_bots])
draw_piechart(pie_chart_labels, pie_chart_values, 'Authors by Gender (last 4 years)')

In [27]:
pie_chart_labels, pie_chart_values = query_total_piechart(INDEX, METRIC_NAME, METRIC_FIELD, 
                                                            [filter_date_1y, filter_merges_addedlines,
                                                             filter_merges_removedlines, filter_bots])
draw_piechart(pie_chart_labels, pie_chart_values, 'Authors by Gender (last year)')

* Evolution and trends of type of contributions (code or others) by gender over time
  * Type of file touched by developers (columns: filetype, gender)

In [28]:
METRIC_NAME = "code_files_touched"
METRIC_FIELD = "id.keyword"
filter_merges_addedlines = Q('range', addedlines={'gt': 0})
filter_merges_removedlines = Q('range', removedlines={'gt': 0})
filter_bots = Q('bool', must_not=[Q('match', gender_analyzed_name='Jenkins')])
filter_filetype = Q('term', filetype='code')
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_merges_addedlines,
                                                               filter_merges_removedlines, filter_bots,
                                                               filter_filetype])

draw_evolutionary_chart("key_as_string", "doc_count")

In [29]:
METRIC_NAME = "others_files_touched"
METRIC_FIELD = "id.keyword"
filter_merges_addedlines = Q('range', addedlines={'gt': 0})
filter_merges_removedlines = Q('range', removedlines={'gt': 0})
filter_bots = Q('bool', must_not=[Q('match', gender_analyzed_name='Jenkins')])
filter_filetype = Q('term', filetype='other')
df = query_metric_over_time(INDEX, METRIC_NAME, METRIC_FIELD, [filter_date_4y, filter_merges_addedlines,
                                                               filter_merges_removedlines, filter_bots,
                                                               filter_filetype])
draw_evolutionary_chart("key_as_string", "doc_count")


# Top Projects by women activity and women population

In [30]:
METRIC_NAME = "commits"
METRIC_FIELD = "id.keyword"

filters = [filter_date_4y, filter_merges_addedlines, filter_merges_removedlines, filter_bots]

s = Search(using=es_conn, index=INDEX)  # Index selection
for filtering in filters:
    s = s.filter(filtering)
s.aggs.bucket('project', 'terms', field='projects', size=1000)\
      .bucket('gender', 'terms', field='gender')\
      .metric(METRIC_NAME, 'cardinality', field=METRIC_FIELD, precision_threshold=10000)\
      .metric('authors', 'cardinality', field='uuid')
result = s.execute()

values = result.to_dict()

In [31]:
value = result.to_dict()["aggregations"]['project']['buckets']
    
df = pandas.DataFrame()
df_summary = pandas.DataFrame(columns=["ratio_authors", "ratio_commits", "authors", "commits", "project"])
for i in value:
    df2 = (pandas.DataFrame.from_dict(i["gender"]["buckets"]))
    df2["project"] = i["key"]
    
    df2[METRIC_NAME] = df2[METRIC_NAME].apply(lambda row:row["value"])
    df2['authors'] = df2['authors'].apply(lambda row:row['value'])
    df = pandas.concat([df, df2])
    
    df_summary2 = pandas.DataFrame(columns=["ratio_authors", "ratio_commits", "authors", "commits", "project"])

    df_summary2["ratio_authors"] = [float(df2[df2["key"]=="female"]["authors"]) / float(df2[df2["key"]=="male"]["authors"])*100]
    df_summary2["ratio_commits"] = [float(df2[df2["key"]=="female"]["commits"]) / float(df2[df2["key"]=="male"]["commits"])*100]
    df_summary2["authors"] = [int(df2[df2["key"]=="female"]["authors"])]
    df_summary2["commits"] = [int(df2[df2["key"]=="female"]["commits"])]
    df_summary2["project"] = [i["key"]]
    df_summary = pandas.concat([df_summary, df_summary2])

    


In [32]:
df_summary

,ratio_authors,ratio_commits,authors,commits,project
0,14.296351,8.693415,576.0,10474.0,Packaging-deb
0,9.653725,4.254443,92.0,1343.0,Infrastructure
0,16.453382,7.250387,90.0,655.0,neutron
0,14.052288,18.943013,43.0,3846.0,fuel
0,20.840065,38.482299,129.0,2424.0,Documentation
0,8.823529,3.717889,9.0,563.0,OpenStack Charms
0,16.152717,7.173331,110.0,723.0,nova
0,20.000000,5.491404,47.0,214.0,Telemetry
0,10.144928,7.988166,35.0,243.0,octavia
0,21.227621,24.959638,83.0,773.0,horizon


In [33]:
df_summary.sort_values("ratio_commits", ascending=False)

,ratio_authors,ratio_commits,authors,commits,project
0,114.285714,176.190476,8.0,37.0,tricircle
0,47.619048,44.444444,10.0,64.0,zun
0,20.840065,38.482299,129.0,2424.0,Documentation
0,21.052632,32.916667,8.0,79.0,kuryr
0,22.807018,27.964602,26.0,790.0,murano
0,39.393939,26.626324,13.0,176.0,dragonflow
0,55.000000,26.506024,11.0,132.0,vitrage
0,12.903226,26.385224,8.0,200.0,congress
0,21.227621,24.959638,83.0,773.0,horizon
0,16.091954,24.457112,42.0,901.0,ironic


In [34]:
df_summary.sort_values("ratio_authors", ascending=False)


,ratio_authors,ratio_commits,authors,commits,project
0,114.285714,176.190476,8.0,37.0,tricircle
0,55.000000,26.506024,11.0,132.0,vitrage
0,47.619048,44.444444,10.0,64.0,zun
0,40.000000,5.042017,6.0,6.0,karbor
0,39.393939,26.626324,13.0,176.0,dragonflow
0,34.883721,6.213873,15.0,129.0,senlin
0,22.807018,27.964602,26.0,790.0,murano
0,22.727273,6.730769,5.0,21.0,winstackers
0,22.330097,8.363971,23.0,91.0,Security
0,21.875000,11.526480,7.0,37.0,RefStack


In [35]:
df_summary.sort_values("authors", ascending=False)


,ratio_authors,ratio_commits,authors,commits,project
0,14.296351,8.693415,576.0,10474.0,Packaging-deb
0,20.840065,38.482299,129.0,2424.0,Documentation
0,16.152717,7.173331,110.0,723.0,nova
0,9.653725,4.254443,92.0,1343.0,Infrastructure
0,16.453382,7.250387,90.0,655.0,neutron
0,21.227621,24.959638,83.0,773.0,horizon
0,13.237640,7.474576,83.0,441.0,Quality Assurance
0,14.228457,9.900091,71.0,436.0,cinder
0,15.476190,10.019776,52.0,456.0,keystone
0,11.037528,4.859577,50.0,308.0,oslo


In [36]:
df_summary.sort_values("commits", ascending=False)


,ratio_authors,ratio_commits,authors,commits,project
0,14.296351,8.693415,576.0,10474.0,Packaging-deb
0,14.052288,18.943013,43.0,3846.0,fuel
0,20.840065,38.482299,129.0,2424.0,Documentation
0,9.653725,4.254443,92.0,1343.0,Infrastructure
0,16.091954,24.457112,42.0,901.0,ironic
0,22.807018,27.964602,26.0,790.0,murano
0,21.227621,24.959638,83.0,773.0,horizon
0,16.152717,7.173331,110.0,723.0,nova
0,9.180328,12.379310,28.0,718.0,Puppet OpenStack
0,16.453382,7.250387,90.0,655.0,neutron


In [37]:
df_summary.sort_values("project", ascending=True)


,ratio_authors,ratio_commits,authors,commits,project
0,16.129032,1.901141,15.0,40.0,Chef OpenStack
0,8.333333,4.741379,3.0,11.0,Community App Catalog
0,20.840065,38.482299,129.0,2424.0,Documentation
0,11.111111,19.298246,1.0,11.0,I18n
0,9.653725,4.254443,92.0,1343.0,Infrastructure
0,8.823529,3.717889,9.0,563.0,OpenStack Charms
0,14.473684,3.612335,22.0,287.0,OpenStackAnsible
0,19.186047,18.381831,33.0,259.0,OpenStackClient
0,14.296351,8.693415,576.0,10474.0,Packaging-deb
0,9.523810,4.280156,2.0,22.0,Packaging-rpm
